In [1]:
def ext_scalars(k, n):
    q = k.order()
    assert gcd(n, q) == 1, "Only multiplicative H90, please!"
    
    phi = cyclotomic_polynomial(n, 'z').change_ring(k)
    
    K = k.extension(n, 'x')
    A = K['z']
    h = A(phi.factor()[0][0])
    
    return K, A.quo(h, 'z')

In [2]:
def frob_l(a, i=1):
    A = a.parent()
    q = a.base_ring().base_ring().order()
    return A(list(c^(q^i) for c in a))

In [3]:
def frob_r(a, i=1):
    A = a.parent()
    z = A.gen()
    q = a.base_ring().base_ring().order()
    return sum(c*z^(j*q^i) for j, c in enumerate(a))

In [4]:
def is_h90(a, zeta):
    return frob_l(a) == zeta * a

In [5]:
def solve_h90(A):
    x = A(1)
    n = A.base_ring().degree()
    z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        x = sum(frob_l(x, i)*z^-i for i in range(n))
    return x

In [104]:
def cyclo_deg(k, n):
    return Zmod(n)(k.characteristic()).multiplicative_order()

def rnorm(a, m):
    A = a.parent()
    n = A.base_ring().degree()
    r = A.degree()
    q = A.base_ring().base_ring().order()
    s = cyclo_deg(GF(q), m)
    d = n // m
    
#    print n, m, r, s, d
    return product(frob_r(a, s*j) for j in range(d))
def toto(k, m, n):
    q = k.order()
    r = cyclo_deg(k, m)
    s = cyclo_deg(k, n)
    t = (q^s-1)//(q^r-1)
    return t, gcd(t, n), n//m

def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]

def testl(p, l, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, l^n)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(l^(n+1))
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(l^(n+1)))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, l^n)
    aa = h1^(l^n)
    bb = i2^(l^n)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(l^(n+1))
    return h1, A2(c.polynomial().list())*h2

def is_iso(h1, h2):
    m = h1.base_ring().degree()
    n = h2.base_ring().degree()
    l = n // m
    i2 = rnorm(h2, m)
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return h1.list()[0].minpoly() == j2.minpoly()  
    
def frob_rc(a, n, m, i=1):
    A = a.parent()
    z = A.gen()
    l = n // m
    u, v = xgcd(l, m)[1:]
    q = a.base_ring().base_ring().order()
    return sum(c*z^(u*l*j)*z^(v*m*j*q^i) for j, c in enumerate(a))

def rnormc(a, m):
    A = a.parent()
    n = a.base_ring().degree()
    q = A.base_ring().base_ring().order()
    l = n // m
    d = cyclo_deg(GF(q), l)
    print d, l
    return product(frob_rc(a, n, m, j) for j in range(d))

"""
Here we find an element h2 solution of H90 in F_p^n compatible with h1.
"""
def findh90(h1, n):
    k = h1.base_ring().base_ring()
    k1, A1 = h1.base_ring(), h1.parent()
    M1 = A1.modulus()
    R = k['z']
    m = k1.degree()
    l = n // m
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(M1.change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^(m)
    bb = i2^(m)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return A2(c.polynomial().list())*h2

def testc(p, m, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, m)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    l = n // m
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^m
    bb = i2^m
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return h1, A2(c.polynomial().list())*h2

In [7]:
K, A = ext_scalars(GF(7), 9)  # 1 = r₁ < r₂ = 2
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v

(True, True, True)

In [29]:
K, A = ext_scalars(GF(19), 9)  # 1 = r₁ = r₂
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v

9 3 1 1 1


(True, True, False)

In [41]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)
v = rnorm(h, 3)
is_h90(u, A.gen()^10), frob_r(u, 3) == u, u == v

15 3 4 1 4


(True, True, True)

In [46]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 4)*frob_r(h, 8)
v = rnorm(h, 5)
is_h90(u, A.gen()^3), frob_r(u, 4) == u, is_h90(v, A.gen()^3), frob_r(v, 4) == v, u == v, u == h^3

15 5 4 4 1


(True, True, False, True, False, True)

In [100]:
K, A = ext_scalars(GF(19), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)
is_h90(u, A.gen()^5), frob_l(u, 3) == u

(False, False)

# An $\ell$-adic tower

In [94]:
p, l, n = 17, 3, 2
cyclo_deg(GF(p), l^n)

2

In [95]:
for i in range(10):
    h1, h2 = testl(p, l, n)
    print is_iso(h1, h2)

True
True
True
True
True
True
True
True
True
True


In [16]:
h2 == h2b

False

In [17]:
is_iso(h1, h2b)

True

In [18]:
h3 = findh90(h2b, 3^3)

In [19]:
is_iso(h1, h3)

True

# Composita

In [113]:
p = 2
k = GF(p)
cyclo_deg(k, 3^2*5)

12

In [114]:
m = 3^1 * 5
n = 3^2 * 5
l = n // m
for i in range(10):
    h1, h2 = testc(p, m, n)
    print is_iso(h1, h2)

True
True
True
True
True
True
True
True
True
True


In [115]:
h1

(x^14 + x^10 + x^7 + x^4 + x^3 + x^2 + x)*z^3 + (x^14 + x^13 + x^10 + x^9 + x^7 + x^5 + x^4 + x^3 + x^2 + 1)*z^2 + (x^14 + x^11 + x^9 + x^8 + x^6 + x^3 + x^2 + x + 1)*z + x^14 + x^13 + x^11 + x^10 + x^9 + x^7 + x^6

In [116]:
h2

(z45^43 + z45^42 + z45^40 + z45^39 + z45^37 + z45^34 + z45^32 + z45^31 + z45^29 + z45^27 + z45^26 + z45^25 + z45^24 + z45^22 + z45^18 + z45^15 + z45^14 + z45^12 + z45^11 + 1)*z^11 + (z45^44 + z45^41 + z45^39 + z45^38 + z45^36 + z45^35 + z45^33 + z45^32 + z45^30 + z45^28 + z45^25 + z45^22 + z45^20 + z45^17 + z45^14 + z45^12 + z45^7 + z45^6 + z45^2 + 1)*z^10 + (z45^44 + z45^43 + z45^42 + z45^38 + z45^37 + z45^36 + z45^29 + z45^27 + z45^26 + z45^24 + z45^23 + z45^20 + z45^17 + z45^15 + z45^13 + z45^12 + z45^10 + z45^9 + z45^8 + z45^6 + z45^4 + z45^3 + z45)*z^9 + (z45^43 + z45^41 + z45^37 + z45^36 + z45^35 + z45^32 + z45^31 + z45^30 + z45^28 + z45^27 + z45^26 + z45^25 + z45^24 + z45^21 + z45^17 + z45^15 + z45^14 + z45^12 + z45^11 + z45^9 + z45^8 + z45^7 + z45^6 + z45^5 + z45^2 + z45)*z^8 + (z45^42 + z45^41 + z45^39 + z45^38 + z45^37 + z45^36 + z45^34 + z45^32 + z45^28 + z45^27 + z45^26 + z45^24 + z45^21 + z45^19 + z45^18 + z45^16 + z45^15 + z45^14 + z45^13 + z45^12 + z45^8 + z45^5 + z45^4 

In [123]:
for x in rnorm(h2, m).minpoly().list():
    print x.minpoly()

x + 1
x^15 + x^7 + 1
x^15 + x^13 + x^8 + x^5 + x^2 + x + 1
x^15 + x + 1
x + 1


In [126]:
for x in h1.minpoly().list():
    print h2.list()[0].parent()(x)

TypeError: unable to coerce <type 'sage.rings.finite_rings.element_givaro.FiniteField_givaroElement'>